# Time Series Modeling for Strawberry Price Prediction

This notebook implements various time series models for strawberry price prediction.

In [1]:
import pandas as pd
import numpy as np
import joblib

from src.fct_feature_eng import *
from src.fct_model import *
from src.parameter import get_dict_params

## 1. Load Processed Data

In [2]:
# Load processed data
train_data = pd.read_csv('data/processed/train_processed.csv')
test_data = pd.read_csv('data/processed/test_processed.csv')

# Convert dates
train_data['start_date'] = pd.to_datetime(train_data['start_date'])
test_data['start_date'] = pd.to_datetime(test_data['start_date'])

# Create time series of the target
train_ts = train_data.set_index('start_date')['price']
test_ts = test_data.set_index('start_date')['price']

# Check for missing values
if train_ts.isnull().any() or test_ts.isnull().any():
    raise ValueError("Missing values detected in processed data")

print("Data loaded successfully:")
print(f"Training set shape: {train_ts.shape}")
print(f"Testing set shape: {test_ts.shape}")

Data loaded successfully:
Training set shape: (210,)
Testing set shape: (47,)


## 2. Train and Save Models

In [3]:
# Load parameters
dict_params = get_dict_params()
model_types = dict_params['model_types']

# Define model functions
model_functions = {
    'naive': lambda train, test: (naive_forecast(train, test), None),
    'arima': fit_arima_model,
    'xgboost': fit_xgboost_model
}

# Train and save models
for model_type in model_types:
    print(f"\nTraining {model_type} model:")
    fit_func = model_functions[model_type]
    pred, model = fit_func(train_ts, test_ts)
    save_model_and_predictions(model, pred, model_type)
    print(f"Completed training {model_type} model")

# Save test data for evaluation
joblib.dump(test_ts, 'models/test_data.joblib')

print("\nAll models have been trained and saved.")


Training naive model:
Completed training naive model

Training arima model:


/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/strawberries-price-prediction-RFzSQChC-py3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/strawberries-price-prediction-RFzSQChC-py3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/strawberries-price-prediction-RFzSQChC-py3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequenc

Completed training arima model

Training xgboost model:
Completed training xgboost model

All models have been trained and saved.


In [13]:
# Code to look for one model :
model_type='arima'
fit_func = model_functions[model_type]
pred, model = fit_func(train_ts, test_ts)
pred

/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/strawberries-price-prediction-RFzSQChC-py3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/strawberries-price-prediction-RFzSQChC-py3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/strawberries-price-prediction-RFzSQChC-py3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequenc

210    3.558085
211    3.197336
212    2.918615
213    2.703268
214    2.536888
215    2.408338
216    2.309018
217    2.232282
218    2.172994
219    2.127186
220    2.091795
221    2.064450
222    2.043324
223    2.027001
224    2.014389
225    2.004645
226    1.997117
227    1.991300
228    1.986806
229    1.983334
230    1.980651
231    1.978579
232    1.976977
233    1.975740
234    1.974784
235    1.974046
236    1.973475
237    1.973034
238    1.972693
239    1.972430
240    1.972227
241    1.972070
242    1.971948
243    1.971855
244    1.971782
245    1.971726
246    1.971683
247    1.971650
248    1.971624
249    1.971604
250    1.971588
251    1.971576
252    1.971567
253    1.971560
254    1.971555
255    1.971550
256    1.971547
Name: predicted_mean, dtype: float64